<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_07_5_custom_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 7: LangChain: Agents**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* Part 7.1: Introduction to LangChain Agents [[Video]](https://www.youtube.com/watch?v=J5Vr___lSSs) [[Notebook]](t81_559_class_07_1_agents.ipynb)
* Part 7.2: Understanding LangChain Agent Tools [[Video]](https://www.youtube.com/watch?v=qMquBmteYw4) [[Notebook]](t81_559_class_07_2_tools.ipynb)
* Part 7.3: LangChain Retrival and Search Tools [[Video]](https://www.youtube.com/watch?v=NB5qGPLoBBE) [[Notebook]](t81_559_class_07_3_search_tools.ipynb)
* Part 7.4: Constructing LangChain Agents [[Video]](https://www.youtube.com/watch?v=OJe5oHvrdHk) [[Notebook]](t81_559_class_07_4_more_agent.ipynb)
* **Part 7.5: Custom Agents** [[Video]](https://www.youtube.com/watch?v=IsJemVYSEdc) [[Notebook]](t81_559_class_07_5_custom_agent.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_experimental langchainhub sentence-transformers

Note: not using Google CoLab


# 7.5: Custom LangChain Agents

In this section, we delve into the creation of custom agents within LangChain. By designing bespoke tools, we can enhance the functionality of our agents to tackle more intricate tasks. These custom agents empower us to tailor solutions to specific requirements, enabling more sophisticated interactions and problem-solving capabilities. By leveraging LangChain's flexible framework, we can seamlessly integrate these custom tools to address unique challenges and optimize performance, providing a more robust and adaptable solution for complex scenarios.

## A Simple Custom Tool

We begin with a very simple tool: a button for the Language Model (LLM) to push. By providing this button, we can observe and confirm that the LLM can indeed interact with it. This fundamental interaction serves as a foundational step, demonstrating the basic capabilities of the LLM to execute commands. The following code constructs an LLM, as we have done previously, allowing us to build upon this basic interaction and progressively incorporate more complex tools and functionalities.

In [2]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )



In the provided code, we create a simple agent tool designed to press a button. We begin by importing the Tool class from the LangChain agents module. The core functionality is defined in the press_button function, which takes a single argument, value. When this function is called, it prints the provided value and returns the string "The button glows red!".

Next, we instantiate a Tool object named button_tool. We specify its name as "button_machine" and provide a description that explains its purpose: it is a big red button that can be pushed, and it accepts a single value when prompted. The func parameter is set to our press_button function, linking the tool's action to our defined behavior. This setup allows the LLM to interact with the button tool by pushing it and observing the corresponding response.

In [3]:
from langchain.agents import Tool

def press_button(value):
  print(f"*************{value}")
  return "The button glows red!"

button_tool = Tool(
    name="button_machine",
    description="A big red button that you can push. You can send a single value to the button, if asked. ",
    func=press_button,
)

In this code, we construct an agent and instruct it to press the button we previously defined. We start by importing the necessary modules and functions, including hub from LangChain, create_tool_calling_agent, and AgentExecutor. We then pull a prompt from the LangChain hub, which serves as the template for our agent's interactions.

We define a list named tools, which includes our previously created button_tool. Using the create_tool_calling_agent function, we create an agent that can utilize the tools we've specified, guided by the pulled prompt. The AgentExecutor class is then used to instantiate an agent_executor, linking the agent with the tools and enabling verbose output for better insight into the interactions.

Finally, we invoke the agent with the input command: "Push the big red button, tell me what happens." This command instructs the agent to use the button tool, demonstrating the agent's ability to perform the defined action and providing feedback on the outcome.

In [4]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [button_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Push the big red button, tell me what happens."})

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5515: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))




> Entering new AgentExecutor chain...

Invoking: `button_machine` with `{'config': {'tags': ['big red button'], 'run_name': 'Push the Big Red Button'}}`


*************{'tags': ['big red button'], 'run_name': 'Push the Big Red Button'}
The button glows red!When you push the big red button, it glows red!

> Finished chain.


{'input': 'Push the big red button, tell me what happens.',
 'output': 'When you push the big red button, it glows red!'}

## Controling a Simple Robot with Tools


In this code, we create a more complex tool involving three buttons that control the movement of a small car. We define a CarTool class that initializes the car's position (x, y) and direction (north, east, south, west). The class also creates a Tool object named control_panel with a description explaining the functions of three buttons: red, green, and yellow.

The CarTool class includes methods to move the car forward (move_forward), turn it left (turn_left), and turn it right (turn_right). The press_button method processes button presses by color. Pressing the red button moves the car forward by one unit in its current direction. Pressing the green button turns the car 90 degrees to the left, and pressing the yellow button turns the car 90 degrees to the right. If an invalid button color is pressed, an error message is returned.

Additionally, the get_position method provides the current position and direction of the car. The example usage at the end demonstrates how to use the CarTool to control the car by pressing different buttons and handling invalid input.

In [5]:
from langchain.tools import StructuredTool
from pydantic import BaseModel
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Define a Pydantic schema for the button press input
class ButtonPressInput(BaseModel):
    button_color: str

class CarTool:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.direction = 'north'
        self.directions = ['north', 'east', 'south', 'west']
        self.tool = StructuredTool(
            name="control_panel",
            description="""Three buttons, red, green and yellow that you can push.
            The red button moves you forward 1 unit.
            The green button turns left 90 degrees,
            the yellow button turns right 90 degrees.""",
            func=self.press_button,
            args_schema=ButtonPressInput  # Define the expected input schema
        )

    def move_forward(self):
        if self.direction == 'north':
            self.y += 1
        elif self.direction == 'east':
            self.x += 1
        elif self.direction == 'south':
            self.y -= 1
        elif self.direction == 'west':
            self.x -= 1

    def turn_left(self):
        current_index = self.directions.index(self.direction)
        self.direction = self.directions[(current_index - 1) % 4]

    def turn_right(self):
        current_index = self.directions.index(self.direction)
        self.direction = self.directions[(current_index + 1) % 4]

    def press_button(self, button_color: str):
        button_color = button_color.strip().lower()
        if button_color == 'red':
            self.move_forward()
            result = "The button glows red, you move forward one unit."
        elif button_color == 'green':
            self.turn_left()
            result = "The button glows green, you turn 90 degrees to the left."
        elif button_color == 'yellow':
            self.turn_right()
            result = "The button glows yellow, you turn 90 degrees to the right."
        else:
            result = "The button buzzes, error."

        print(f"Current car position: {self.get_position()}")
        return result

    def get_position(self):
        return self.x, self.y, self.direction

# Example usage
car_tool = CarTool()

try:
    print(car_tool.tool.func('red'))  # Move forward
    print(car_tool.tool.func('green'))  # Turn left
    print(car_tool.tool.func('yellow'))  # Turn right
    print(car_tool.tool.func('blue'))  # Invalid button color, raises ValueError
except ValueError as e:
    print(e)


Current car position: (0, 1, 'north')
The button glows red, you move forward one unit.
Current car position: (0, 1, 'west')
The button glows green, you turn 90 degrees to the left.
Current car position: (0, 1, 'north')
The button glows yellow, you turn 90 degrees to the right.
Current car position: (0, 1, 'north')
The button buzzes, error.


We now construct an agent and provide this tool, we request the car to move in a rectangle.

In [6]:
# Initialize LLM and create an agent
MODEL = 'gpt-4o-mini'
llm = ChatOpenAI(model=MODEL, temperature=0.2, n=1)

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [car_tool.tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Invoke the agent to control the car
agent_executor.invoke({"input": "Push the buttons in a way that causes the car to move in a rectangle."})




/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `control_panel` with `{'button_color': 'red'}`


Current car position: (0, 2, 'north')
The button glows red, you move forward one unit.
Invoking: `control_panel` with `{'button_color': 'green'}`


Current car position: (0, 2, 'west')
The button glows green, you turn 90 degrees to the left.
Invoking: `control_panel` with `{'button_color': 'red'}`


Current car position: (-1, 2, 'west')
The button glows red, you move forward one unit.
Invoking: `control_panel` with `{'button_color': 'green'}`


Current car position: (-1, 2, 'south')
The button glows green, you turn 90 degrees to the left.
Invoking: `control_panel` with `{'button_color': 'red'}`


Current car position: (-1, 1, 'south')
The button glows red, you move forward one unit.
Invoking: `control_panel` with `{'button_color': 'green'}`


Current car position: (-1, 1, 'east')
The button glows green, you turn 90 degrees to the left.
Invoking: `control_panel` with `{'button_color': 're

{'input': 'Push the buttons in a way that causes the car to move in a rectangle.',
 'output': 'The car has moved in a rectangle as follows:\n\n1. Moved forward 1 unit.\n2. Turned 90 degrees to the left.\n3. Moved forward 1 unit.\n4. Turned 90 degrees to the left.\n5. Moved forward 1 unit.\n6. Turned 90 degrees to the left.\n7. Moved forward 1 unit.\n8. Turned 90 degrees to the left.\n\nThe car is now back at its starting position, having completed a rectangular path.'}